### 1. Download Shopify dataset and create table script:

In [ ]:
!gsutil cp gs://cs327e-open-access/spanner_shopify.zip /home/jupyter/snippets

In [ ]:
!unzip /home/jupyter/snippets/spanner_shopify.zip

### 2. Configure Spanner CLI and create Shopify Database

In [ ]:
%env CLI=/home/jupyter/spanner-cli-0.9.9/spanner-cli
%env SPAN_PROJECT=cs327e-fa2021
%env SPAN_INSTANCE=span
%env SPAN_DATABASE=span

In [ ]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

In [ ]:
CLI_CONNECT="$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE"

In [ ]:
!{CLI_CONNECT} -e "create database shopify"

In [ ]:
%env SPAN_DATABASE=shopify

In [ ]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

In [ ]:
!{CLI_CONNECT} -f /home/jupyter/snippets/spanner_shopify/create_tables.sql

In [ ]:
!{CLI_CONNECT} -e "show tables"

In [ ]:
!{CLI_CONNECT} -e "show create table key_benefits"

In [ ]:
!{CLI_CONNECT} -e "show create table apps"

In [ ]:
!{CLI_CONNECT} -e "show create table pricing_plans"

In [ ]:
!{CLI_CONNECT} -e "show create table apps_categories"

In [ ]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

In [ ]:
!{CLI_CONNECT} -e "show create table categories"

In [ ]:
!{CLI_CONNECT} -e "show create table reviews"

### 3. Populate tables

In [ ]:
from google.cloud import spanner
from google.cloud.spanner_v1 import param_types
import math, pandas as pd

In [ ]:
span_instance='span'
span_database='shopify'
dir_path = '/home/jupyter/snippets/spanner_shopify'
spanner_client = spanner.Client()
instance = spanner_client.instance(span_instance)
database = instance.database(span_database)

In [ ]:
def split_dataframe(df, split_size): 
    splits = list()
    num_splits = math.ceil(len(df) / split_size) 
    for i in range(0, len(df), split_size):
        df_subset = df.iloc[i:i + split_size]
        splits.append(df_subset)
    return splits

In [ ]:
table = 'apps'
df = pd.read_csv(dir_path + '/apps.csv', sep=',', header=0, lineterminator='\n')

df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'url', 'title', 'developer', 'developer_link', 'icon', 'rating', 'reviews_count'),
            values=rows
        )

In [ ]:
table = 'apps_categories'
df = pd.read_csv(dir_path + '/apps_categories.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'category_id'),
            values=rows
        )

In [ ]:
table = 'categories'
df = pd.read_csv(dir_path + '/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()
print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
with database.batch() as batch:
    batch.insert(
        table=table,
        columns=('id', 'title'),
        values=rows
    )

In [ ]:
table = 'key_benefits'
df = pd.read_csv(dir_path + '/key_benefits.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'title', 'description'),
            values=rows
        )

In [ ]:
table = 'pricing_plans'
df = pd.read_csv(dir_path + '/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'app_id', 'title', 'price'),
            values=rows
        )

In [ ]:
table = 'pricing_plan_features'
df = pd.read_csv(dir_path + '/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('pricing_plan_id', 'app_id', 'feature'),
            values=rows
        )

In [ ]:
!{CLI_CONNECT} -e "select count(*) from apps"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from apps_categories"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from categories"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from key_benefits"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from pricing_plans"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from pricing_plan_features"

In [ ]:
def insert_app(transaction):

    row_ct = transaction.execute_update(
        "INSERT INTO apps (id, url, title, developer, developer_link, icon, reviews_count) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'https://apps.shopify.com/watchlist?surface_detail=inventory-management&surface_inter_position=1&surface_intra_position=9&surface_type=category'," 
          "'Back in Stock Product Alerts', "
          "'Swym Corporation', "
          "'https://apps.shopify.com/partners/developer-ca6a967f09890f68',"
          "'https://apps.shopifycdn.com/listing_images/9905a4c8f22cb4a3b0c32af55a58ec21/icon/e6d46a7e5e1df375d542d033aae80459.png?height=72&width=72',"
           "0)"
    )
    
    print("{} apps record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO apps_categories (app_id, category_id) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '737ad50051083aa051d127a53b3ac0da')"
    )
    
    print("{} apps_categories record(s) inserted.".format(row_ct))
    
    
    row_ct = transaction.execute_update(
        "INSERT INTO key_benefits (app_id, title, description) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'Drive Back In Stock Sales', 'Back in stock, out of stock, pre order & restock alerts')"
    )
    
    print("{} key_benefits record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plans (id, app_id, title, price) "
        "VALUES ('961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', '683d06af-14c7-4733-9bde-ec5b699af996', "
                "'Free Trial', 14.99)"
    )
    
    print("{} pricing_plans record(s) inserted.".format(row_ct))
    

    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plan_features (app_id, pricing_plan_id, feature) "
        " VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', "
                 "'Up to 250 alert requests/mo')"
    )
    
    print("{} pricing_plan_features record(s) inserted.".format(row_ct))
      
database.run_in_transaction(insert_app)

In [ ]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE pricing_plans "
            "SET price = price / 2 "
            "WHERE price > 0"
        )

    print("{} pricing_plans record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

In [ ]:
def delete_app(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM apps WHERE id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )

        print("{} apps record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM apps_categories WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} apps_categories record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM key_benefits WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} key_benefits record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plans WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plans record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plan_features WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plan_features record(s) deleted.".format(row_ct))

database.run_in_transaction(delete_app)

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "show create table apps_categories"

In [ ]:
!{CLI_CONNECT} -e "CREATE INDEX apps_rating_idx ON apps(rating)"